#  PYNQ 2DCONV

 Engineer:  :  Clancy lee
 
 NAME     : PYNQ Z1
 
 Data     : 2018.10.17
 
 School    : jiang nan university
 
 Design Name: 2DCONV --pynq z1
 
 Description : HLS Synthesis Code

In [1]:
#==================================================
#                Load overlay
#=================================================
from pynq import Overlay
overlay = Overlay("./2dconv.bit")
overlay?

In [2]:
#==================================================
#                Load 2DConv IP
#=================================================
sobel_ip=overlay.HW_2DConv_Mmap_3_0
overlay.ip_dict

{'HW_2DConv_Mmap_3_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'HW_2DConv_Mmap_3_0',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:hls:HW_2DConv_Mmap_3:1.0'}}

In [3]:
#==================================================
#                allocated  memory
#=================================================
from pynq import Xlnk
import numpy as np
   
ROWS_O =270
COLS_O =480
FILTER_SIZE    = 3
STRIDE         = 1
ROWS_I         =  (ROWS_O*STRIDE+FILTER_SIZE-1)#272
COLS_I         = (COLS_O*STRIDE+FILTER_SIZE-1)#482

DATA_IN_SHMM_SIZE = (ROWS_I*COLS_I+FILTER_SIZE*FILTER_SIZE)#272*482+9
DATA_OUT_SHMM_SIZE = (ROWS_O*COLS_O)#270*480
IP_CONFIG_SHMM_SIZE  = 128


xlnk = Xlnk()
############## clean up #############
xlnk.xlnk_reset()

#allocated the memory  inbuff
data_in = xlnk.cma_array(shape=(DATA_IN_SHMM_SIZE,), dtype=np.int8)#272*482+9
data_in2 = xlnk.cma_array(shape=(DATA_IN_SHMM_SIZE,), dtype=np.int8)#272*482+9
DATA_IN_SHMM_BASE_ADDR= data_in.physical_address
DATA_IN_SHMM_BASE_ADDR2= data_in2.physical_address

#allocated the memory  outbuff
data_out = xlnk.cma_array(shape=(DATA_OUT_SHMM_SIZE,), dtype=np.int8)
data_out2 = xlnk.cma_array(shape=(DATA_OUT_SHMM_SIZE,), dtype=np.int8)
DATA_OUT_SHMM_BASE_ADDR=data_out.physical_address
DATA_OUT_SHMM_BASE_ADDR2=data_out2.physical_address


print(DATA_IN_SHMM_BASE_ADDR)
print(DATA_IN_SHMM_BASE_ADDR2)
print(DATA_OUT_SHMM_BASE_ADDR)
print(DATA_OUT_SHMM_BASE_ADDR2)


378011648
378273792
377880576
378535936


In [4]:
#==================================================
#                wrtie data to data_in 
#=================================================
import numpy as np
FILTER_SIZE = 3
filter = [[-1,-1,-1],[0,0,0],[1,1,1]]
filter2 = [[-1,0,1],[-1,0,1],[-1,0,1]]
data_in_temp = np.zeros(DATA_IN_SHMM_SIZE,dtype = np.int8)
data_in_temp2 = np.zeros(DATA_IN_SHMM_SIZE,dtype = np.int8)
print("Initialize input data......\n")
for i in range (0,ROWS_I):
    for j in range(0,COLS_I):
        if(i<ROWS_O and j<COLS_O):
            data_in_temp[i*COLS_I+j]  = j
            data_in_temp2[i*COLS_I+j]  = j
        else: 
            data_in_temp[i*COLS_I+j] =  0    #pixels out of boundary are filled with 0
            data_in_temp2[i*COLS_I+j] =  0
np.copyto(data_in,data_in_temp)
np.copyto(data_in2,data_in_temp2)
print("Completed!\n")
np.savetxt('data_in.txt',data_in)
np.savetxt('data_in2.txt',data_in2)

#input_tmp_img = np.loadtxt('yolo_input_tmp_mem_1.txt', dtype= 'uint')

Initialize input data......

Completed!



In [5]:
#==================================================
#               filter
#=================================================
print("Filter:\n")
for  i in  range(0,FILTER_SIZE):
    for j in  range(0,FILTER_SIZE):
            data_in[ROWS_I*COLS_I + i*FILTER_SIZE+j] = filter[i][j]
            data_in2[ROWS_I*COLS_I + i*FILTER_SIZE+j] = filter2[i][j]


Filter:



In [6]:
#==================================================
#           [FPGA] Run 2D convolution hw Ex
#=================================================
from pynq import MMIO
import time
IP_BASE_ADDRESS        =  0x43C00000
ADDRESS_RANGE          =  1000
DATA_IN_BASE_ADDR_REG  =  0x10
DATA_IN_BASE_ADDR_REG2  =  0x18
DATA_OUT_BASE_ADDR_REG =  0x20
DATA_OUT_BASE_ADDR_REG2 =  0x28
AP_CTRL_REG            =  0x00
AP_START               =  0x01
AP_DONE                =  0x02
mmio = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE)
start = time.time()
mmio.write(DATA_IN_BASE_ADDR_REG,DATA_IN_SHMM_BASE_ADDR)
mmio.write(DATA_IN_BASE_ADDR_REG2,DATA_IN_SHMM_BASE_ADDR2)
mmio.write(DATA_OUT_BASE_ADDR_REG,DATA_OUT_SHMM_BASE_ADDR)
mmio.write(DATA_OUT_BASE_ADDR_REG2,DATA_OUT_SHMM_BASE_ADDR2)
mmio.write(AP_CTRL_REG,AP_START)

while True:
    if(mmio.read(AP_CTRL_REG)&AP_DONE!=AP_DONE):    
        break

stop = time.time()
time_2dconv_fpga = stop - start
print("2Dconv FPGA time:",time_2dconv_fpga)
print("FPGA Accelerate Completed!\n")
np.savetxt('data_out.txt',data_out)
np.savetxt('data_out2.txt',data_out2)
data_out
data_out2

2Dconv FPGA time: 0.0027091503143310547
FPGA Accelerate Completed!



ContiguousArray([12, 12, 12, ...,  4, 68, 66], dtype=int8)